In [25]:
from osgeo import gdal
from osgeo import ogr
import os

path = os.getcwd() + '\Dados vetoriais\Pontos amostrais_Piracicaba\Dados_227pts_original_06_03_2021.shp'


pts = ogr.Open(path, 0)
layer = pts.GetLayer()


In [26]:

path2 = os.getcwd() + '\Dados vetoriais\Piracicaba\piraciacaba.tif'
piracicaba = gdal.Open(path2)
gt = piracicaba.GetGeoTransform()

In [27]:
ulx = gt[0] #coordenada X do canto esquerdo de cima
uly = gt[3] #coordenada Y do canto esquerdo de cima
res = gt[1] #resolução/tamanho do pixel

xsize = piracicaba.RasterXSize
ysize = piracicaba.RasterYSize

lrx = ulx + xsize * res
lry = uly - ysize * res

piracicaba = None
pts = layer = None

# nearest neighbor interpolation    
nn = gdal.Grid("nearest_r10.tif", path, zfield="Argila",
               algorithm = "nearest:radius1=10000:radius2=10000", outputBounds = [ulx,uly,lrx,lry],
               width = xsize, height = ysize)
nn = None

# moving average
ma = gdal.Grid("average_r10.tif", path, zfield="Argila",
               algorithm = "average:radius1=10000:radius2=10000:angle=0",
               outputBounds = [ulx,uly,lrx,lry],
               width = xsize, height = ysize)
ma = None

# inverse distance to a power
idw = gdal.Grid("invdist_sm50r20.tif", path, zfield = "Argila",
                algorithm = "invdist:power=4:radius1=20000:radius2=20000:smoothing=100",
                outputBounds = [ulx,uly,lrx,lry],
                width = xsize, height = ysize)
idw = None

# linear interpolation
lin = gdal.Grid("linear.tif", path, zfield = "Argila",
                algorithm = "linear",
                outputBounds = [ulx,uly,lrx,lry],
                width = xsize, height = ysize)
lin = None

# inverse distance to a power with nearest neighbor
idwnn = gdal.Grid("invdistnn_r10.tif", path, zfield = "Argila",
                algorithm = "invdistnn:power=4:radius=10000:smoothing=50",
                outputBounds = [ulx,uly,lrx,lry],
                width = xsize, height = ysize)
idwnn = None